In [11]:
import pandas as pd
import re
import openpyxl

# Load the Excel file
df = pd.read_excel('Desktop\smc.xlsx')
df.head()

# Deleting unnecessary columns
columns_to_delete = ['Headline', 'Opening Text', 'Country', 'Subregion', 'Language', 'Reach', 
                     'Desktop Reach', 'Mobile Reach', 'Twitter Social Echo', 'Facebook Social Echo', 
                     'Reddit Social Echo', 'National Viewership', 'Engagement', 'AVE', 'Sentiment', 
                     'Key Phrases', 'Input Name', 'Keywords', 'Twitter Authority', 'Tweet Id', 
                     'Twitter Id', 'Twitter Client', 'Alternate Date Format', 'State', 'City', 
                     'Social Echo Total', 'Editorial Echo', 'Views', 'Estimated Views', 'Likes', 
                     'Replies', 'Retweets', 'Comments', 'Shares', 'Reactions', 'Threads', 
                     'Document Tags', 'Document ID', 'Parent URL']

# Deleting columns
df.drop(columns=columns_to_delete, inplace=True, errors='ignore')

# Save the new DataFrame back to Excel
df.to_excel(r'desktop\smc.xlsx', index=False, sheet_name='Data')

# Removing quotes
if 'Hit Sentence' in df.columns:
    # Remove rows where 'Hit Sentence' contains 'QT'
    df = df[~df['Hit Sentence'].str.contains('QT', case=False)]
    
    # Display resulting DataFrame
    print(df.head())
else:
    print("Error: 'Quotes' does not exist in the Content.")
df.head()

# Remove unnecessary things from Account names like "modi ka pariwar" and some accounts which we don't need like satnam singh chd
# Define the list of Twitter screen names to exclude
twitter_screen_names_to_exclude = ['Modi Ka Pariwar', 'Satnam Singh Sandhu', 'Modi ka Parivar']

# Generate the condition dynamically
condition = df['Twitter Screen Name'].str.contains('|'.join(twitter_screen_names_to_exclude), case=False)

# Filter rows based on conditions
df = df[~(condition & (df['Twitter Followers'] < 10))]

# Save the modified DataFrame back to Excel
df.to_excel(r'desktop\smc.xlsx', index=False, sheet_name='Data')
df.head()

# Remove URLs containing bjp keywords
df = df[~df['URL'].str.contains('bjp|rss|news', case=False)]

# Removing extremely political words from content and unnecessary content 
if 'Hit Sentence' in df.columns:
    patterns_to_remove = ['QT', 'RT', 'BJP', 'Amit Shah', 'NaMo', '#ModiAgainIn2024', 'Congress', '#ElectoralBondScam', 'NaMo', '#AbkiBaar400Paar', '#MereVikasKaDoHisaab', 'Shahjahan Seikh', 'BJP', 'bharat ratna', 'Narasimha Rao']
    
    # Compile regex patterns for case-insensitive matching
    regex_patterns = [re.compile(f'\\b{re.escape(pattern)}\\b', flags=re.IGNORECASE) for pattern in patterns_to_remove]
    
    for pattern in regex_patterns:
        try:
            # Use regex pattern to find matches in 'Hit Sentence' column
            df = df[~df['Hit Sentence'].str.contains(pattern)]
        except KeyError:
            print(f"Error: 'Hit Sentence' column does not exist in the DataFrame.")

    # Display the resulting DataFrame after removing rows
    print(df.head())
else:
    print("Error: 'Hit Sentence' column does not exist in the DataFrame.")
    df.head()

# Remove rows with less than 15 words in the content/ Remove one liners
df = df[df['Hit Sentence'].str.split().apply(len) >= 15]
df.head()


# check if the Account's bio is not right
word_list = ['अध्य', 'भाजपा', 'सेवक', 'विधानसभा', 'मंत्री', 'विधायक', 'Karyakarta', 'Secretary', 'offi', 
             'member', 'minist', 'MP', 'MLA', 'Gover', 'Modi', 'news', 'जनसामान्य', 'कार्य', 'President', 
             'सर्वो', 'Justiceforssr', 'BJYM', 'journa', 'भारतीय जनता पार्टी', 'bjp', 'gpt', 'Chancello', 
             'आधिकारिक', 'खब', 'Mayor', 'Jan Ki Baat', 'सांसद', 'control', 'tube', 'प्रभात', 'Tamasha', 
             'जय हिंद जय भारत', 'PIB', 'Skilled', 'न्यूज़', 'MyGov', 'Info', 'TV', 'PMO', 'Minister', 
             'Parliament', 'Media', 'Rajya Sabha', 'drm', 'Maan Ki Baat', 'Railway', 'Official', 'Union', 
             'Bjp', 'RSS', 'Swayam', 'IT Cell', 'CSIS', 'firm', 'फाउंडेशन', 'Politician', '#HamaraAppNaMoApp']


# Fill missing values in 'Twitter Bio' column with empty strings
df['Twitter Bio'].fillna('', inplace=True)

# Exclude rows containing the words from the word list in the "Twitter Bio" column
for word in word_list:
    df = df[~df['Twitter Bio'].str.contains(word, na=False, case=False)]

# Display the updated DataFrame
df.head()


# Removing multiple tags (@) in the content 
if 'Hit Sentence' in df.columns:
    # Count the occurrences of '@' in each row of the 'Hit Sentence' column
    at_count_series = df['Hit Sentence'].apply(lambda x: x.count('@'))
    
    # Filter rows where '@' count is not greater than 1
    df = df[at_count_series <= 1]

    # Display the resulting DataFrame after removing rows
    print("Final DataFrame after removing rows:")
    print(df.head())
else:
    print("Error: 'Hit Sentence' column does not exist in the DataFrame.")

# Remove quotes
if 'Hit Sentence' in df.columns:
    # Remove rows where 'Hit Sentence' contains 'QT'
    df = df[~df['Hit Sentence'].str.contains('QT', case=False)]
    
    # Display resulting DataFrame
    print(df.head())
else:
    print("Error: 'Hit Sentence' column does not exist in the DataFrame.")

# Political hashtag we need 
# Define the hashtags and keywords to filter
hashtag_to_filter = ['#GoaModiKaParivar', '#AbkiBaar400Paar']

# Create the combined regex pattern to filter for any of the hashtags or keywords
combined_pattern = '|'.join(hashtag_to_filter)

# Filter the DataFrame for the specified hashtags and keywords
filtered_df = df[df['Hit Sentence'].str.contains(combined_pattern, case=False)]

# Check if any rows are filtered
if not filtered_df.empty:
    # Sort the filtered DataFrame by number of followers in descending order
    sorted_df = filtered_df.sort_values(by='Twitter Followers', ascending=False)
    
    # Take only the top three posts
    top_three_posts = sorted_df.head(5)
    
    # Drop other rows
    df = df.drop(filtered_df.index)
    
    # Concatenate the top three posts DataFrame with the original DataFrame
    df = pd.concat([df, top_three_posts], ignore_index=True)
    
    # Display the top three posts
    print("Top three posts for the hashtag", hashtag_to_filter)
    print(top_three_posts)
else:
    print(f"No posts found for the hashtag {hashtag_to_filter}.")

df.head()


df = df[df['Hit Sentence'].str.contains('@narendramodi', na=False) | ~df['Hit Sentence'].str.contains('@')]


# Sort the DataFrame by 'Twitter Followers' in descending order and 'Is Verified' in ascending order
df.sort_values(by=['Twitter Followers', 'Is Verified'], ascending=[False, True], inplace=True)

# Remove rows where the 'Hit Sentence' column starts with three dots
df = df[~df['Hit Sentence'].str.startswith('...', na=False)]

# Save the sorted and filtered DataFrame back to the same sheet in Excel
with pd.ExcelWriter(r'Desktop\smc.xlsx', engine='openpyxl', mode='a') as writer:
    df.to_excel(writer, index=False, sheet_name='Sorted_Data', startrow=0, startcol=0)

print("Filtered, sorted, and updated data has been saved to 'smc.xlsx'")


# Drop duplicate values in the 'Hit Sentence' column
df_unique = df.drop_duplicates(subset=['Hit Sentence'])

# Display the DataFrame with unique values
print(df_unique)


# Define the filename for the filtered data
filtered_filename = 'Desktop\smc.xlsx'

# Save the filtered DataFrame to Excel
df.to_excel(filtered_filename, index=False)

print(f"Filtered data has been saved to {filtered_filename}")


                  Date                                                URL  \
0  01-Apr-2024 09:31AM  https://twitter.com/nakkheeranweb/statuses/177...   
1  01-Apr-2024 12:16PM  https://twitter.com/KantipurTVHD/statuses/1774...   
2  01-Apr-2024 12:59PM  https://twitter.com/Ravisutanjani/statuses/177...   
3  01-Apr-2024 02:05AM  https://twitter.com/dtnoorkhan/statuses/177453...   
4  01-Apr-2024 02:05AM  https://twitter.com/dtnoorkhan/statuses/177453...   

                                        Hit Sentence   Source      Influencer  \
0  "நாட்டில் ஊழலுக்கு எதிராக பெரும் போரை மேற்கொண்...  Twitter  @nakkheeranweb   
1  hilly and rural districts before the beginning...  Twitter   @kantipurtvhd   
2  PM Modi Addresing RBI’s 90th Anniversary Event...  Twitter  @ravisutanjani   
3  پر ہے انڈیا میں پاکستانیوں اللہ کا شکر ادا کرو...  Twitter     @dtnoorkhan   
4  پر ہے انڈیا میں پاکستانیوں اللہ کا شکر ادا کرو...  Twitter     @dtnoorkhan   

  Twitter Screen Name                   User Profi